# import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os


sys.path.append('..')
from model.util_model import DictManager
from data_process.util_data import calculate_improvement_ratio


%matplotlib qt
plt.rcParams['font.family'] = 'Times new Roman'  # Use a serif font family
plt.rcParams['font.size'] = 14


def plot_metric_curve(data:np.array, last_index:int=-20):
    min_index = np.argmin(data)
    min_value = data[min_index]
    plt.axhline(y=min_value, color='gray', linestyle='--', linewidth=1)
    data_index = np.array(range(len(data)))
    plt.plot(data_index, data, 'black')
    plt.plot(data_index[last_index:], data[last_index:], 'blue', label='Ave Value: {:.2f}'.format(np.mean(data[last_index:])))
    plt.plot(min_index, min_value, 'ro', label='Min Value: {:.2f}'.format(min_value))
    plt.legend()


def get_reg_from_dict(dictManager:DictManager, filename, origin_colums, filter_columns=[]):
    tmp = dictManager.load_dict(filename)['test']['val']
    df = pd.DataFrame(columns=['name', 'MAE', 'SAE', 'MAE(0)', 'MAE(1)'])
    df['name'] = origin_colums
    df['MAE'] = tmp['MAE(apps)'].squeeze()
    df['SAE'] = tmp['SAE(apps)'].squeeze()
    df['MAE(0)'] = tmp['MAE(offon)'].squeeze()[0]
    df['MAE(1)'] = tmp['MAE(offon)'].squeeze()[1]
    if len(filter_columns) > 0:
        filter_df = df[df['name'].isin(filter_columns)]
        return filter_df
    else:
        return df


def list_files_without_extension(directory):
    file_names = set()
    for root, _, files in os.walk(directory):
        for file in files:
            file_name, _ = os.path.splitext(file)
            file_names.add(file_name)
    return sorted(file_names,reverse=True)



redd_house_1_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9', 'washer_dryer-10', 'microwave-11',
                'bathroom_gfi-12', 'electric_heat-13', 'stove-14', 'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']
redd_apps_to_train = ['refrigerator-5', 'dishwaser-6', 'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12']
redd_index = [redd_house_1_cols.index(element) for element in redd_apps_to_train]



ukdale_house_1_cols = [
 'boiler-2',
 'solar_thermal_pump-3',
 'laptop-4',
 'washing_machine-5',
 'dishwasher-6',
 'tv-7',
 'kitchen_lights-8',
 'htpc-9',
 'kettle-10',
 'toaster-11',
 'fridge-12',
 'microwave-13',
 'lcd_office-14',
 'hifi_office-15',
 'breadmaker-16',
 'amp_livingroom-17',
 'adsl_router-18',
 'livingroom_s_lamp-19',
 'soldering_iron-20',
 'gigE_&_USBhub-21',
 'hoover-22',
 'kitchen_dt_lamp-23',
 'bedroom_ds_lamp-24',
 'lighting_circuit-25',
 'livingroom_s_lamp2-26',
 'iPad_charger-27',
 'subwoofer_livingroom-28',
 'livingroom_lamp_tv-29',
 'DAB_radio_livingroom-30',
 'kitchen_lamp2-31',
 'kitchen_phone&stereo-32',
 'utilityrm_lamp-33',
 'samsung_charger-34',
 'bedroom_d_lamp-35',
 'coffee_machine-36',
 'kitchen_radio-37',
 'bedroom_chargers-38',
 'hair_dryer-39',
 'straighteners-40',
 'iron-41',
 'gas_oven-42',
 'data_logger_pc-43',
 'childs_table_lamp-44',
 'childs_ds_lamp-45',
 'baby_monitor_tx-46',
 'battery_charger-47',
 'office_lamp1-48',
 'office_lamp2-49',
 'office_lamp3-50',
 'office_pc-51',
 'office_fan-52',
 'LED_printer-53']
ukdale_apps_to_train = ['laptop-4', 'washing_machine-5', 'kettle-10', 'toaster-11', 'fridge-12', 'microwave-13']
ukdale_index = [ukdale_house_1_cols.index(element) for element in ukdale_apps_to_train]


fig_save_dir = r'C:\Users\29492\Desktop\server_data\plot'

In [ ]:
df = get_reg_from_dict(DictManager(r'C:\Users\29492\Downloads'), 'REDD_Model_Combined_all_0109-1', redd_house_1_cols, redd_apps_to_train)
print(df['MAE'].mean())

# Main

## REDD

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\main_REDD'
dm = DictManager(file_save_dir)

In [ ]:
df = pd.DataFrame(columns=['name', 'MAE', 'SAE', 'MAE(0)', 'MAE(1)'])
for app in redd_apps_to_train:
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='SGN',appname=app,date='0422')
    tmp = dm.load_dict(filename)['test']['val']
    print(f'--{app}--')
    print(tmp)
    new_row = {'name':app,
               'MAE':tmp['MAE(apps)'].squeeze(),
               'SAE':tmp['SAE'].squeeze(),
               'MAE(0)':tmp['MAE(offon)'].squeeze()[0],
               'MAE(1)':tmp['MAE(offon)'].squeeze()[1]}
    df = df.append(new_row, ignore_index=True)
    # print(f'--{app}--')
    # for key, value in tmp.items():
    #     print(key, value.squeeze())

df

In [ ]:
for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='Final',appname='all',date=f'1228-{i}')
    df = get_reg_from_dict(dm,filename, redd_house_1_cols)
    print(df)
    print(df['MAE'].mean(),df['SAE'].mean())


## UKDALE

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\main_UKDALE'
dm = DictManager(file_save_dir)

In [ ]:
df = pd.DataFrame(columns=['name', 'MAE', 'SAE', 'MAE(0)', 'MAE(1)'])
for app in ukdale_apps_to_train:
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='S2P',appname=app,date='1231')
    tmp = dm.load_dict(filename)['test']['val']
    print(f'--{app}--')
    print(tmp)
    new_row = {'name':app,
               'MAE':tmp['MAE(apps)'].squeeze(),
               'SAE':tmp['SAE'].squeeze(),
               'MAE(0)':tmp['MAE(offon)'].squeeze()[0],
               'MAE(1)':tmp['MAE(offon)'].squeeze()[1]}
    df = df.append(new_row, ignore_index=True)
    # print(f'--{app}--')
    # for key, value in tmp.items():
    #     print(key, value.squeeze())

df

In [ ]:

for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='Final',appname='all',date=f'0101-{i}')
    df = get_reg_from_dict(dm,filename, ukdale_house_1_cols, ukdale_apps_to_train)
    print(df)
    print(df['MAE'].mean(),df['SAE'].mean())

# Sub

## ablation

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_ablation'
dm = DictManager(file_save_dir)

### check

In [ ]:
# REDD_AblationMark_classification_all_0103-0
for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_classification',appname='all',date=f'0103-{i}')
    tmp = dm.load_dict(filename)['test']['val']
    df = pd.DataFrame(columns=['name', 'Acc'])
    df['name'] = redd_house_1_cols
    df['Acc'] = tmp['Acc(apps)'].squeeze()
    filter_df = df[df['name'].isin(redd_apps_to_train)]
    print(filter_df)
    print(filter_df['Acc'].mean())

for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_Ws_Mtl',appname='all',date=f'0103-{i}')
    tmp = dm.load_dict(filename)['test_clas']['val']
    df = pd.DataFrame(columns=['name', 'Acc'])
    df['name'] = redd_house_1_cols
    df['Acc'] = tmp['Acc(apps)'].squeeze()
    filter_df = df[df['name'].isin(redd_apps_to_train)]
    print(filter_df)
    print(filter_df['Acc'].mean())

In [ ]:
df = get_reg_from_dict(dm,'REDD_AblationMark_Ws_Mtl_Alw_all_0104-AWL1L-1', redd_house_1_cols, redd_apps_to_train)
print(df)
print(df['MAE'].mean())

In [ ]:
'''
AblationMark_baseline
AblationMark_Ws
AblationMark_Ws_Mtl
AblationMark_Ws_Mtl_Alw
AblationMark_Ws_Mtl_Alw_SGNs
AblationMark_Ws_Mtl_Alw_SGNh
'''

for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_Ws_Mtl_Alw_SGNs',appname='all',date=f'0104-{i}')
    tmp = dm.load_dict(filename)['test']['val']
    df = pd.DataFrame(columns=['name', 'MAE', 'SAE', 'MAE(0)', 'MAE(1)'])
    df['name'] = redd_house_1_cols
    df['MAE'] = tmp['MAE(apps)'].squeeze()
    df['SAE'] = tmp['SAE(apps)'].squeeze()
    df['MAE(0)'] = tmp['MAE(offon)'].squeeze()[0]
    df['MAE(1)'] = tmp['MAE(offon)'].squeeze()[1]

    filter_df = df[df['name'].isin(redd_apps_to_train)]
    print(filter_df)
    print(filter_df['MAE'].mean())

In [ ]:
fig, axs = plt.subplots(1,2)
for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_Ws_Mtl_Alw',appname='all',date=f'0103-AWL{1}L-{i}')
    tmp = dm.load_dict(filename)
    data_reg = tmp['train_vali']['train']['MAE(apps)'].squeeze()[:,redd_index].mean(axis=1)
    data_clas = tmp['train_vali_clas']['train']['Acc(apps)'].squeeze()[:,redd_index].mean(axis=1)
    axs[0].plot(data_reg,label=f'{i}')
    axs[1].plot(data_clas,label=f'{i}')

axs[0].legend()
axs[1].legend()

In [ ]:
fig, axs = plt.subplots(1,2)
for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_Ws_Mtl_Alw',appname='all',date=f'0103-AWL{2}L-{i}')
    tmp = dm.load_dict(filename)
    data_reg = tmp['train_vali']['train']['MAE(apps)'].squeeze()[:,redd_index].mean(axis=1)
    data_clas = tmp['train_vali_clas']['train']['Acc(apps)'].squeeze()[:,redd_index].mean(axis=1)
    axs[0].plot(data_reg,label=f'{i}')
    axs[1].plot(data_clas,label=f'{i}')

axs[0].legend()
axs[1].legend()

In [ ]:
for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='AblationMark_Ws_Mtl_Alw',appname='all',date=f'0104-AWL{1}L-{i}')
    tmp = dm.load_dict(filename)['test']['val']
    df = pd.DataFrame(columns=['name', 'MAE', 'SAE', 'MAE(0)', 'MAE(1)'])
    df['name'] = redd_house_1_cols
    df['MAE'] = tmp['MAE(apps)'].squeeze()
    df['SAE'] = tmp['SAE(apps)'].squeeze()
    df['MAE(0)'] = tmp['MAE(offon)'].squeeze()[0]
    df['MAE(1)'] = tmp['MAE(offon)'].squeeze()[1]

    filter_df = df[df['name'].isin(redd_apps_to_train)]
    print(filter_df)
    print(filter_df['MAE'].mean())

### ALW TRAINING

In [ ]:
files = ['REDD_AblationMark_Ws_Mtl_all_0103-1','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-2','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL2L-0']
result_reg = []
result_clas = []

for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['train_vali']['train']['MAE(apps)'].squeeze()[:,redd_index].mean(axis=1)
    data_clas = tmp['train_vali_clas']['train']['Acc(apps)'].squeeze()[:,redd_index].mean(axis=1)
    result_reg.append(data_reg)
    result_clas.append(data_clas)


labels = ['direct summation','1-layer ALW','2-layer ALW']
fig1, ax1 = plt.subplots(figsize=(5, 5))
for d,l in zip(result_reg, labels):
    ax1.plot(d,label=l)
ax1.set_ylim(20, 70)
ax1.legend()
# ax1.set_title('Impact of auto loss weighting on training process')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Mean absolute error')
# fig1.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_training_MAE.pdf'), format="pdf", bbox_inches='tight')

fig2, ax2 = plt.subplots(figsize=(5, 5))
for d,l in zip(result_clas, labels):
    ax2.plot(d,label=l)
ax2.set_ylim(0.96, 1)
ax2.legend()
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuary')
# fig2.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_training_Acc.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
result_reg = []
result_clas = []
for filename_prefix in ['REDD_AblationMark_Ws_Mtl_all_0109-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL1L-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL2L-']:
    tmp_reg = []
    tmp_clas = []
    for i in range(10):
        filename = filename_prefix + f'{i}'
        tmp = dm.load_dict(filename)
        data_reg = tmp['train_vali']['train']['MAE(apps)'].squeeze()[:,redd_index].mean(axis=1)
        data_clas = tmp['train_vali_clas']['train']['Acc(apps)'].squeeze()[:,redd_index].mean(axis=1)
        tmp_reg.append(data_reg)
        tmp_clas.append(data_clas)
    tmp_reg = np.array(tmp_reg)
    tmp_clas = np.array(tmp_clas)
    result_reg.append(tmp_reg)
    result_clas.append(tmp_clas)
result_reg = np.array(result_reg)
result_clas = np.array(result_clas)

In [ ]:
for i in range(3):
    d = result_reg[i]
    df = pd.DataFrame(d)
    plt.figure(figsize=(4, 4))
    df.boxplot()
    mean_values = df.mean()
    plt.plot(np.arange(len(mean_values))+1,mean_values, label='Mean', color='red', marker='o', linestyle='-')

    y_mean = mean_values[-5:].mean()  # 设定横线的y轴值
    plt.axhline(y=y_mean, color='red', linestyle='-')
    plt.text(1, y_mean+0.5, f'{y_mean:.2f}', verticalalignment='bottom', color='green')

    y_min = d[:,-5:].min()  # 设定横线的y轴值
    plt.axhline(y=y_min, color='black', linestyle='--')
    plt.text(1, y_min-3.8, f'{y_min:.2f}', verticalalignment='bottom', color='black')

    plt.xticks(ticks=np.arange(5,51,5)-1, labels=np.arange(5,51,5))

    plt.ylim(10, 70)
    # plt.ylim(0.96, 1)
    
    plt.ylabel("Mean absolute error")
    plt.xlabel("Epoch")
    plt.show()

    plt.savefig(os.path.join(fig_save_dir,f'exp_sub_ALW_training_MAE_{i}.pdf'), format="pdf", bbox_inches='tight')

    example_array = d[:,-5:]

    # 计算四分位数
    q1 = np.percentile(example_array, 25)  # 第一四分位数（25%）
    mean = np.mean(example_array)
    q3 = np.percentile(example_array, 75)  # 第三四分位数（75%）

    print(q1, mean, q3)
    # break

In [ ]:
for i in range(3):
    d = result_clas[i]
    df = pd.DataFrame(d)
    plt.figure(figsize=(4, 4))
    df.boxplot()
    mean_values = df.mean()
    plt.plot(np.arange(len(mean_values))+1,mean_values, label='Mean', color='red', marker='o', linestyle='-')

    y_mean = mean_values[-5:].mean()  # 设定横线的y轴值
    plt.axhline(y=y_mean, color='red', linestyle='-')
    plt.text(1, y_mean-0.003, f'{y_mean:.4f}', verticalalignment='bottom', color='green')

    y_min = d[:,-5:].max()  # 设定横线的y轴值
    plt.axhline(y=y_min, color='black', linestyle='--')
    plt.text(1, y_min+0.0002, f'{y_min:.4f}', verticalalignment='bottom', color='black')
    plt.xticks(ticks=np.arange(5,51,5)-1, labels=np.arange(5,51,5))

    # plt.ylim(10, 70)
    plt.ylim(0.975, 1)

    plt.ylabel("Accuary")
    plt.xlabel("Epoch")
    plt.show()

    plt.savefig(os.path.join(fig_save_dir,f'exp_sub_ALW_training_Acc_{i}.pdf'), format="pdf", bbox_inches='tight')
    example_array = d[:,-5:]

    # 计算四分位数
    q1 = np.percentile(example_array, 25)  # 第一四分位数（25%）
    mean = np.mean(example_array)
    q3 = np.percentile(example_array, 75)  # 第三四分位数（75%）

    print(q1, mean, q3)
    # break

In [ ]:
result_reg = []
for filename_prefix in ['REDD_AblationMark_Ws_Mtl_all_0109-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL1L-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL2L-']:
    tmp_reg = []
    tmp_clas = []
    for i in range(10):
        filename = filename_prefix + f'{i}'
        tmp = dm.load_dict(filename)
        data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
        tmp_reg.append(data_reg)
    tmp_reg = np.array(tmp_reg)
    result_reg.append(tmp_reg)
result_reg = np.array(result_reg)

In [ ]:
for i in range(3):
    d = result_reg[i]
    df = pd.DataFrame(d)
    plt.figure(figsize=(4, 4))
    df.boxplot()

    # plt.xticks(ticks=np.arange(5,51,5)-1, labels=np.arange(5,51,5))

    plt.ylim(20, 50)
    # plt.ylim(0.96, 1)

    plt.ylabel("Accuary")
    plt.xlabel("Epoch")
    plt.show()

    # plt.savefig(os.path.join(fig_save_dir,f'exp_sub_ALW_training_Acc_{i}.pdf'), format="pdf", bbox_inches='tight')
    example_array = d[:,-5:]

    # 计算四分位数
    q1 = np.percentile(example_array, 25)  # 第一四分位数（25%）
    mean = np.mean(example_array)
    q3 = np.percentile(example_array, 75)  # 第三四分位数（75%）

    print(q1, mean, q3)

### ALW test

In [ ]:
result_reg = []
result_clas = []
for filename_prefix in ['REDD_AblationMark_Ws_Mtl_all_0109-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL1L-', 'REDD_AblationMark_Ws_Mtl_Alw_all_0109-AWL2L-']:
    tmp_reg = []
    tmp_clas = []
    for i in range(10):
        filename = filename_prefix + f'{i}'
        tmp = dm.load_dict(filename)
        data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
        data_clas = tmp['test_clas']['val']['Acc(apps)'].squeeze()[redd_index]
        tmp_reg.append(data_reg)
        tmp_clas.append(data_clas)
    result_reg.append(np.array(tmp_reg))
    result_clas.append(np.array(tmp_clas))
result_reg = np.array(result_reg)
result_clas = np.array(result_clas)

In [ ]:
matrix_data = result_reg.mean(axis=1)
labels_dim1 = ['Direct summation','Automatic loss \nweighting (1-layer)','Automatic loss \nweighting (2-layer)']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(6, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')
plt.ylabel('Mean absolute error')
plt.ylim(10,50)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='lower center', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_result_mae.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
matrix_data = result_clas.mean(axis=1)
labels_dim1 = ['Direct summation','Automatic loss \nweighting (1-layer)','Automatic loss \nweighting (2-layer)']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(6, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average Accuracy')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value, f'{value:.4f}', ha='center', va='bottom')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Accuary')
# plt.ylim(10,50)
plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='lower center', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_result_acc.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
files = ['REDD_AblationMark_Ws_Mtl_all_0103-1','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL2L-1','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-2']
# files = ['REDD_AblationMark_Ws_Mtl_all_0103-1','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-0','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-2']
result_reg = []
result_clas = []

for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
    data_clas = tmp['test_clas']['val']['Acc(apps)'].squeeze()[redd_index]
    result_reg.append(data_reg)
    result_clas.append(data_clas)

result_reg = np.array(result_reg)
result_clas = np.array(result_clas)
print(result_reg)
print(result_reg.mean(axis=1))
print(result_clas)
print(result_clas.mean(axis=1))

In [ ]:
matrix_data = result_reg
labels_dim1 = ['direct summation','1-layer ALW','2-layer ALW']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
# plt.ylim(0,50)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='lower left', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_result_mae.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
matrix_data = result_clas
labels_dim1 = ['direct summation','1-layer ALW','2-layer ALW']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Accuary')
# plt.ylim(0,50)
plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='lower left', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_ALW_result_acc.pdf'), format="pdf", bbox_inches='tight')

### SGN

In [ ]:
files = ['REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-2','REDD_AblationMark_Ws_Mtl_Alw_SGNh_all_0103-2','REDD_AblationMark_Ws_Mtl_Alw_SGNs_all_0103-2']
result_reg = []
result_clas = []

for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
    data_clas = tmp['test_clas']['val']['Acc(apps)'].squeeze()[redd_index]
    result_reg.append(data_reg)
    result_clas.append(data_clas)

result_reg = np.array(result_reg)
result_clas = np.array(result_clas)
print(result_reg)
print(result_reg.mean(axis=1))
print(result_clas)
print(result_clas.mean(axis=1))


matrix_data = result_reg
labels_dim1 = ['w/o SGM','hard SGM','soft SGM']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_SGN_overall.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
files = ['REDD_AblationMark_baseline_all_0103-0','REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL1L-2','REDD_AblationMark_Ws_Mtl_Alw_SGNh_all_0103-1','REDD_AblationMark_Ws_Mtl_Alw_SGNs_all_0103-0']
result_on = []
result_off = []

for f in files:
    tmp = dm.load_dict(f)['test']['val']['MAE(offon)'].squeeze()

    result_off.append(tmp[0,redd_index])
    result_on.append(tmp[1,redd_index])

result_on = np.array(result_on)
result_off = np.array(result_off)
print(result_on)
print(result_on.mean(axis=1))
print(result_off)
print(result_off.mean(axis=1))

In [ ]:
matrix_data = result_off
labels_dim1 = ['baseline','w/o SGM','hard SGM','soft SGM']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,50)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_SGN_off.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
matrix_data = result_on
labels_dim1 = ['baseline','w/o SGM','hard SGM','soft SGM']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,1500)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='upper center', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_SGN_on.pdf'), format="pdf", bbox_inches='tight')

In [ ]:


matrix_data = result_reg
labels_dim1 = ['no gated mechanism','hard SGN','soft SGN']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')
# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)

## model structure

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_model_structure'
dm = DictManager(file_save_dir)

In [ ]:
# Final Model_Deeper Model_Complex Model_Combined

for i in range(3):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='Model_Deeper',appname='all',date=f'0112-{i}')
    df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
    print(df)
    print(df['MAE'].mean(),df['SAE'].mean())

In [ ]:
files = ['REDD_Final_all_1228-0',
         'REDD_Model_Deeper_all_0112-2',
         'REDD_Model_complex_all_0101-0', 
         'REDD_Model_Combined_all_0109-2']
result_reg = []

for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
    result_reg.append(data_reg)

result_reg = np.array(result_reg)
print(result_reg)
print(result_reg.mean(axis=1))

In [ ]:
matrix_data = result_reg
labels_dim1 = ['Vanilla','Deeper','Enhanced','Refined']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 4))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
# for i, value in enumerate(line_values):
#     plt.text(i*bar_width*7 + bar_width*matrix_data.shape[1]*0.4, value+0.5, f'{value:.2f}', ha='center', va='bottom')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')

# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,45)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_model_structure.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
'''
Vanilla 411MB 17s/epoch
Model_Deeper 78.3MB 11s/epoch
Model_complex 1.59GB 122s/epoch
Model_Combined 560MB 42s/epoch
'''

model_size = [411, 78.3, 1628, 560] # MB
model_time = [element*50/60 for element in [17, 11, 122, 42]] # minute
labels_x = ['Vanilla','Deeper','Enhanced','Refined']
x = np.arange(len(labels_x))

fig, ax1 = plt.subplots(figsize=(5, 4))

bar1 = ax1.bar(x - 0.1, model_size, width=0.2, color='b', label='size')
# ax1.set_xlabel(labels_x)
ax1.set_ylabel('Model parameter size (Mb)', color='b')
ax1.tick_params('y', colors='b')
ax1.legend()

# 创建第二个 Y 轴
ax2 = ax1.twinx()
bar2 = ax2.bar(x + 0.1, model_time, width=0.2, color='r', label='time')
ax2.set_ylabel('Model training  time (minute)', color='r')
ax2.tick_params('y', colors='r')
ax2.legend()

ax1.set_xticks(x, labels_x)

bars = [bar1, bar2]
labels = [bar.get_label() for bar in bars]
ax1.legend(bars, labels, loc='upper left')
ax2.legend().set_visible(False)
# 显示图例
fig.tight_layout()

plt.show()
plt.savefig(os.path.join(fig_save_dir,'exp_sub_model_structure_cost.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
calculate_improvement_ratio(vanilla_score=17.65, neo_score=[15.041, 12.03, 12.45], psr=-1)

## data amount

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_dataamount'
dm = DictManager(file_save_dir)

num_days= 21
score_list = []
for num_day in range(num_days):
    tmp_list = []
    for i in range(3):
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='Final',appname='all',date=f'dataamount-0106-{num_day}-{i}')
        df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
        tmp_list.append(df['MAE'].mean())
    score_list.append(tmp_list)
score_list = np.array(score_list)
score_mark = score_list

score_list = []
for num_day in range(num_days):
    tmp_list = []
    for i in range(3):
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='baseline',appname='all',date=f'dataamount-0106-{num_day}-{i}')
        df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
        tmp_list.append(df['MAE'].mean())
    score_list.append(tmp_list)
score_list = np.array(score_list)
score_baseline = score_list

In [ ]:
score_mark.mean(axis=1)

In [ ]:
score_baseline.mean(axis=1)

In [ ]:
print(score_mark.mean(axis=1)[5-1], score_baseline.mean(axis=1)[20-1])

In [ ]:
calculate_improvement_ratio(vanilla_score=27.05, neo_score=[19.51], psr=-1)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(5, 5))
x = np.arange(num_days)+1


for d, l in zip([score_mark, score_baseline], ['ours', 'baseline']):
    tmp = d.mean(axis=-1)
    ax1.plot(x, tmp, label=l)

ax1.legend()
ax1.set_xlabel('Amount of data')
ax1.set_ylabel('Mean absolute error')

In [ ]:
fig1, ax1 = plt.subplots(figsize=(5, 5))
x = np.arange(num_days)+1

tmp = score_mark.mean(axis=-1)
ax1.plot(x, tmp, label='ours')

tmp = score_baseline.mean(axis=-1)
ax1.plot(x, tmp, label='baseline')

ax1.legend()
ax1.set_xlabel('Amount of data')
ax1.set_ylabel('Mean absolute error')

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 5))
x = np.arange(num_days)+1


for d, l,s in zip([score_mark, score_baseline], ['ours', 'baseline'], ['+','x']):

    tmp_x = [item for item in x for _ in range(3)]
    tmp_y = d.flatten()
    ax1.scatter(tmp_x, tmp_y, label=l, marker=s)

    tmp = d.mean(axis=-1)
    degree = 7
    coefficients = np.polyfit(x, tmp, degree)
    poly_fit = np.poly1d(coefficients)
    fit_y_values = poly_fit(x)
    ax1.plot(x, fit_y_values, label=l+'(fitting)')

ax1.legend()
ax1.set_xlabel('Amount of data (day)')
ax1.set_ylabel('Mean absolute error')
ax1.set_xticks(x)
fig1.savefig(os.path.join(fig_save_dir,'exp_sub_dataamount.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
score_mark[0]

## mislabel

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_mislabel'
dm = DictManager(file_save_dir)

In [ ]:
'''
mislabel_accurate_to_minute
mislabel_random_shift
mislabel_event_missing
mislabel_minute_missing
mislabel_shift_missing
'''
for i in range(5):
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='Final',appname='all',
                                                           date=f'0109-mislabel_shift_missing-{i}')
    df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
    print(df)
    print(df['MAE'].mean(),df['SAE'].mean())

In [ ]:
files = ['REDD_Final_all_0106-0',
         'REDD_Final_all_0106-1',
         'REDD_Final_all_0106-2']
result_reg = []
for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
    result_reg.append(data_reg)

print(result_reg)
print(np.array(result_reg).mean(axis=1))

In [ ]:
strategy = ['mislabel_accurate_to_minute',
    'mislabel_random_shift',
    'mislabel_event_missing',
    'mislabel_minute_missing',
    'mislabel_shift_missing']

result_reg = []

files = ['REDD_AblationMark_baseline_all_0103-0',
         'REDD_Final_all_1228-0']
for f in files:
    tmp = dm.load_dict(f)
    data_reg = tmp['test']['val']['MAE(apps)'].squeeze()[redd_index]
    result_reg.append(data_reg)



for s in strategy:
    tmp = []
    for i in range(5):
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model='Final',appname='all',
                                                            date=f'0109-{s}-{i}')
        df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
        tmp.append(df['MAE'])
    tmp = np.array(tmp)
    result_reg.append(tmp.mean(axis=0))

result_reg = np.array(result_reg)
print(result_reg)

In [ ]:
matrix_data = result_reg
labels_dim1 = ['Baseline','Vanilla'
               ,'Low time granularity','Randomized time drift', 'Unexpected event missing'
               , 'Low time granularity\n+Unexpected event missing', 'Randomized time drift\n+Unexpected event missing']
labels_dim1 = ['baseline','ours (accurate label)'
               ,'ours (MS1)','ours (MS2)', 'ours (MS3)'
               , 'ours (MS1+3)', 'ours (MS2+3)']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(12, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
# for i, value in enumerate(line_values):
#     plt.text(i*bar_width*7 + bar_width*matrix_data.shape[1]*0.4, value+0.5, f'{value:.2f}', ha='center', va='bottom')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')

# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_mislabel.pdf'), format="pdf", bbox_inches='tight')

## app num

In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_appnum'
dm = DictManager(file_save_dir)

index = range(5,31)
score_list = []
for num in index:
    tmp_list = []
    for i in range(3):
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='Final',appname='all',
                                                               date=f'appnum-0111-{num}-{i}')
        dict = dm.load_dict(filename)['test']['val']['MAE(apps)']
        tmp_list.append(np.squeeze(dict)[0:5].mean())
    score_list.append(tmp_list)
score_list = np.array(score_list)
score_mark = score_list

score_list = []
for num in index:
    tmp_list = []
    for i in range(3):
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='baseline',appname='all',
                                                               date=f'appnum-0111-{num}-{i}')
        dict = dm.load_dict(filename)['test']['val']['MAE(apps)']
        tmp_list.append(np.squeeze(dict)[0:5].mean())
    score_list.append(tmp_list)
score_list = np.array(score_list)
score_baseline = score_list

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 5))
x = index


for d, l,s in zip([score_mark, score_baseline], ['ours', 'baseline'], ['+','x']):

    tmp_x = [item for item in x for _ in range(3)]
    tmp_y = d.flatten()
    ax1.scatter(tmp_x, tmp_y, label=l, marker=s)

    tmp = d.mean(axis=-1)
    degree = 7
    coefficients = np.polyfit(x, tmp, degree)
    poly_fit = np.poly1d(coefficients)
    fit_y_values = poly_fit(x)
    ax1.plot(x, fit_y_values, label=l+'(fitting)')

ax1.legend()
ax1.set_xlabel('Number of appliances')
ax1.set_ylabel('Mean absolute error')
ax1.set_xticks(x)
fig1.savefig(os.path.join(fig_save_dir,'exp_sub_apppnum.pdf'), format="pdf", bbox_inches='tight')

## weak vs strong


In [ ]:
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_weak_vs_strong'
dm = DictManager(file_save_dir)

In [ ]:
file_names = list_files_without_extension(file_save_dir)
for file_name in file_names:
    print(file_name)

In [ ]:
'''
5
REDD_AblationMark_Ws_all
REDD_AblationMark_Ws_Mtl_all
REDD_AblationMark_Ws_Mtl_Alw_all
REDD_AblationMark_Ws_Mtl_Alw_Sgns_all
3
REDD_AblationMark_Ss_all
REDD_AblationMark_Ss_Mtl_all
REDD_AblationMark_Ss_Mtl_Alw_all
REDD_AblationMark_Ss_Mtl_Alw_Sgns_all
'''
df_all = pd.DataFrame(columns=['MAE', 'SAE'])
for i in range(5):
    filename = '{prefix}_{date}'.format(prefix='REDD_AblationMark_Ss_Mtl_Alw_Sgns_all', date=f'0420-{i}')
    df = get_reg_from_dict(dm,filename, redd_house_1_cols, redd_apps_to_train)
    print(df)
    print(df['MAE'].mean(),df['SAE'].mean())
    df_all = df_all.append({'MAE':df['MAE'].mean(), 'SAE':df['SAE'].mean()}, ignore_index=True)

print(10*'-')
print(df_all)

In [ ]:
result_Ws_SS = []

colname = 'MAE'
files = ['REDD_AblationMark_Ws_all_0419-2',
         'REDD_AblationMark_Ws_Mtl_all_0419-2',
         'REDD_AblationMark_Ws_Mtl_Alw_all_0419-4',
         'REDD_AblationMark_Ws_Mtl_Alw_Sgns_all_0419-0']
result_reg = []
for f in files:
    df = get_reg_from_dict(dm, f, redd_house_1_cols, redd_apps_to_train)
    print(df[colname].mean())
    result_reg.append(df[colname])
result_Ws = np.array(result_reg)

files = ['REDD_AblationMark_Ss_all_0419-2',
         'REDD_AblationMark_Ss_Mtl_all_0419-0',
         'REDD_AblationMark_Ss_Mtl_Alw_all_0420-1',
         'REDD_AblationMark_Ss_Mtl_Alw_Sgns_all_0420-4']
result_reg = []
for f in files:
    df = get_reg_from_dict(dm, f, redd_house_1_cols, redd_apps_to_train)
    print(df[colname].mean())
    result_reg.append(df[colname])
result_SS = np.array(result_reg)

In [ ]:
matrix_data = result_Ws
labels_dim1 = ['W/O','AL','AL+ALW','AL+ALW+SGM']
labels_dim2 = [element.split('-')[0].replace('_',' ') for element in redd_apps_to_train]
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(6, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

line_values = matrix_data.mean(axis=1)
plt.plot(index + bar_width*matrix_data.shape[1]*0.4, line_values, marker='o', linestyle='-', color='black', label='Average MAE')
# for i, value in enumerate(line_values):
#     plt.text(i*bar_width*7 + bar_width*matrix_data.shape[1]*0.4, value+0.5, f'{value:.2f}', ha='center', va='bottom')
for i, value in enumerate(line_values):
    plt.text(index[i] + bar_width*2.0, value + 0.5, f'{value:.2f}', ha='center', va='bottom')

# plt.title('Grouped Bar Chart Example')
# plt.xlabel('Categories')
plt.ylabel('Mean absolute error')
plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=2)
plt.savefig(os.path.join(fig_save_dir,'exp_sub_supervision_weak.pdf'), format="pdf", bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'exp_sub_supervision_strong.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
matrix_data = result_Ws_SS
labels_dim1 = ['W/O','AL','AL+\nALW','AL+\nALW+\nSGM']
labels_dim2 = ['Weakly-supervised', 'Strongly-supervised']
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.15
index = np.arange(num_rows)

fig = plt.figure(figsize=(5, 5))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

plt.ylabel('Mean absolute error')
# plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=1)
# plt.savefig(os.path.join(fig_save_dir,'exp_sub_weak_vs_strong.pdf'), format="pdf", bbox_inches='tight')

# calculate ratio

In [ ]:
calculate_improvement_ratio(vanilla_score=29.624, neo_score=[43.564,36.05,31.752,16.944], psr=-1)

In [ ]:
calculate_improvement_ratio(vanilla_score=11.87, neo_score=[14], psr=-1)

In [ ]:
calculate_improvement_ratio(vanilla_score=7.08, neo_score=[8.35], psr=-1)

In [ ]:
calculate_improvement_ratio(vanilla_score=29.20, neo_score=[42.43], psr=-1)

In [ ]:
calculate_improvement_ratio(vanilla_score=41.65, neo_score=[17.64], psr=-1)

In [ ]:
calculate_improvement_ratio(vanilla_score=5.09, neo_score=[4.71], psr=-1)

In [ ]:
Vanilla 411MB 17s/epoch
Model_Deeper 78.3MB 11s/epoch
Model_complex 1.59GB 122s/epoch
Model_Combined 560MB 42s/epoch

# label plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

# 生成示例数据
x = np.linspace(0, 10, 100)  # 生成 0 到 10 之间的 100 个数据点作为 x 值

# 输入数据（示例）
input_data = np.sin(x)  # 使用正弦函数生成示例输入数据
# 实际值（示例）
ground_truth = np.cos(x)  # 使用余弦函数生成示例实际值
# 预测值（示例）
prediction = np.sin(x) + np.random.normal(0, 0.1, size=x.shape)  # 使用带有噪声的正弦函数生成示例预测值

# 创建折线图
plt.figure(figsize=(10, 6))
plt.plot(x, input_data, c='g', label='总功耗')
plt.plot(x, ground_truth, c='black', label='负荷功耗真实值')
plt.plot(x, prediction, c='b', label='负荷功耗预测值')

# 添加标题和图例
plt.legend(bbox_to_anchor=(0.5, 1.1), loc='upper center', ncol=3)

# 显示图表
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# plt.savefig('label.pdf', format="pdf", bbox_inches='tight')

In [ ]:
# in-domain redd
matrix_data = np.array([[41.652	,21.542	,23.63,	17.644,	13.32],[24.542,	14.036	,13.652	,11.85	,10.366]]).T
# cross-domain redd
# matrix_data = np.array([[24.07, 	19.65 ,	19.97 ,	14.00 	,11.87 ],[16.34 ,	16.72 	,16.12 	,12.20 	,10.24 ]]).T
# in-domain redd
# matrix_data = np.array([[26.63 ,	18.58 ,	11.48, 	9.35 ,	7.18 ],[15.57 ,	14.32 ,	6.60 ,	7.40 ,	4.60 ]]).T
# matrix_data = np.array([[23.03 ,	16.17 ,	9.46, 	5.09 ,	4.71 ],[15.32 ,	14.35 ,	5.81 ,	4.14 ,	2.99 ]]).T
# in-domain redd
# matrix_data = np.array([[23.63 ,	15.95 ,	13.45 ,	13.14, 	10.73 ],[19.01 ,	12.98 ,	10.91 ,	11.47 ,	9.64 ]]).T
# matrix_data = np.array([[21.18 ,	12.37 ,	11.33 ,	8.35, 	7.08 ],[18.17 ,	11.22 ,	9.10 ,	7.79 ,	6.62 ]]).T
labels_dim1 = ['DAE','S2S','S2P','Ours','SGN']
labels_dim2 = ['MAE','SAE']
patterns = ['/', '\\', '|', '-', '+']
num_rows, num_cols = matrix_data.shape
bar_width = 0.3
index = np.arange(num_rows)

fig = plt.figure(figsize=(6, 4))
for i in range(num_cols):
    plt.bar(index + i * bar_width, matrix_data[:, i], width=bar_width, label=labels_dim2[i])

plt.ylabel('Performance score')
# plt.ylim(0,60)
# plt.ylim(0.88,1)
plt.xticks(index + (bar_width * (num_cols - 1) / 2), labels_dim1)  # 居中显示 x 轴标签
plt.tight_layout()
plt.legend(loc='best', ncol=1)
# plt.savefig(os.path.join(fig_save_dir,'main_models_in_redd_mae.pdf'), format="pdf", bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_cross_redd_mae.pdf'), format="pdf", bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_in_ukdale_mae.pdf'), format="pdf", bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_cross_ukdale_mae.pdf'), format="pdf", bbox_inches='tight')

plt.savefig(os.path.join(fig_save_dir,'main_models_in_redd_mae.png'), bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_cross_redd_mae.png'), bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_in_ukdale_mae.png'), bbox_inches='tight')
# plt.savefig(os.path.join(fig_save_dir,'main_models_cross_ukdale_mae.png'), bbox_inches='tight')